# НЕФТЯНАЯ СКВАЖИНА

В проекте необходимо определить наиболее выгодный регион нефтедобычи.

В качестве исходных данных в 3 регионах изучено 10 000 месторождений, где измерили качество нефти и объём её запасов. Необходимо построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Для анализа будет использован *Bootstrap.* В данных представлены основне значимые признаки, определяющие емкость скважины (f0, f1, f2).

План работы над проектом:
- Загрузка и подготовка данных
- Обучение и проверка модели
- Подготовка к расчету прибыли
- Расчет прибыли и рисков
- Выводы по проекту

## Загрузка и подготовка данных

In [1]:
#импорт библиотек
import pandas as pd
import plotly as plt
import matplotlib.pyplot as plt
import numpy as np
#!pip install plotly_express
#импортируем необходимые библиотеки
import plotly.express as px
pd.set_option('display.max_columns', 300)
%matplotlib inline
from scipy import stats as st
from math import factorial
import math
from scipy import stats as st


from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

import random

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score 
from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error


In [2]:
#выгрузка данных
df_0 = pd.read_csv('C:/DATA/geo_data_0.csv')
df_1 = pd.read_csv('C:/DATA/geo_data_1.csv')
df_2 = pd.read_csv('C:/DATA/geo_data_2.csv')

In [3]:
#оценим информацию в таблицах

display(df_0.head(5))
df_0.info()

display(df_1.head(5))
df_1.info()

display(df_2.head(5))
df_2.info()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


На первый взгляд каких-либо проблем в данных нет. В каждом городе по 10000 значений, нулевые значения отсутствуют. Избавимся от дубликатов, если они есть:

In [4]:
df_0 = df_0.drop_duplicates().reset_index(drop=True)
df_1 = df_1.drop_duplicates().reset_index(drop=True)
df_2 = df_2.drop_duplicates().reset_index(drop=True)

Модель должна быть обучена отдельно для каждого из регионов. Соответственно, подготовим для них целевые признаки и обучающие признаки отдельно:

In [5]:
features_0 = df_0.drop(['id', 'product'], axis=1)
target_0 = df_0['product']

features_1 = df_1.drop(['id', 'product'], axis=1)
target_1 = df_1['product']

features_2 = df_2.drop(['id', 'product'], axis=1)
target_2 = df_2['product']

## Обучение и проверка модели

 Модель призвана определять количественную характеристику - объем запасов в скважине. Опыт подсказывает, что наибольшая точность модели обеспечивается случайным регрессионным лесом. Соответственно, такую модель мы и будем использовать в качестве основной. 

In [6]:
#опробуем обучение для нулевого региона
#разобьем выборки в соответствии с заданием:

features_train_0, features_valid_0, target_train_0, target_valid_0 = train_test_split(
    features_0, target_0, test_size=.25 , random_state=12345)

In [ ]:
#определим наилучшие гиперпараметры модели:

best_model = None
best_r2 = 0
best_depth = 0
best_est = 0
for est in range(1, 20):
    for depth in range (1, 29):
        model = RandomForestRegressor(random_state=12345, n_estimators=est, max_depth=depth) 
        model.fit(features_train_0, target_train_0)
        
        result = model.score(features_valid_0, target_valid_0)
        print (result)
           
        if result > best_r2:
            best_r2 = result
            best_depth = depth
            best_model = model
            best_est = est

print("Наилучший коэффициент множественной детерминации:", best_r2, "Количество деревьев:", best_est, "Максимальная глубина:", depth)
print('RMSE:', mean_squared_error())

Дабы не обучать модель многократно каждый раз, зафиксируем наилучшие параметры:

Лучшая доля правильных ответов 0.2968821496528229 Количество деревьев: 19 Максимальная глубина: 28
Проверим, какие результаты покажет линейная регрессия:

In [7]:
features_train_0, features_valid_0, target_train_0, target_valid_0 = train_test_split(
    features_0, target_0, test_size=.25 , random_state=12345)

model = LinearRegression()
model.fit(features_train_0, target_train_0)
predictions_valid = model.predict(features_valid_0)
print("R2 на валидационной выборке: ", model.score(features_valid_0, target_valid_0)) 
print("RMSE", mean_squared_error(target_valid_0, predictions_valid) ** 0.5) 

R2 на валидационной выборке:  0.27994321524487786
RMSE 37.5794217150813


Не смотря на то, что случайный лес показал лучшие результаты, заказчиком установлено использование линейной регрессии. Соответственно, далее будет использована модель линейной регрессии.

In [8]:
#напишем функцию, обучающую модель и рассчитывающую интересующие нас характеристики
def geo_model(features, target):
    features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=.25 , random_state=12345)
    
    #model = RandomForestRegressor(n_estimators=19, max_depth=28, random_state=123456 )
    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    
    return predictions_valid, mean_squared_error(target_valid, predictions_valid) ** 0.5, target_valid

In [9]:
#Зафиксируем необходимые данные: предсказания, RMSE, правильные ответы
prediction_valid_0, rmse_0, target_valid_0 = geo_model(features_0, target_0)
prediction_valid_1, rmse_1, target_valid_1  = geo_model(features_1, target_1)
prediction_valid_2, rmse_2, target_valid_2  = geo_model(features_2, target_2)



In [10]:
print ('Средний запас предсказанного сырья в регионе 0:', prediction_valid_0.mean(), ', а RMSE -', rmse_0)
print ('Средний запас предсказанного сырья в регионе 1:', prediction_valid_1.mean(), ', а RMSE -', rmse_1)
print ('Средний запас предсказанного сырья в регионе 2:', prediction_valid_2.mean(), ', а RMSE -', rmse_2)

Средний запас предсказанного сырья в регионе 0: 92.59256778438035 , а RMSE - 37.5794217150813
Средний запас предсказанного сырья в регионе 1: 68.72854689544602 , а RMSE - 0.8930992867756168
Средний запас предсказанного сырья в регионе 2: 94.96504596800489 , а RMSE - 40.02970873393434


Результаты показывают, что для нулевого и второго региона, где в среднем запасы достаточно большие, оказывается велик и RMSE. Т.е. в среднем модель ошибается примерно на 40%, что не удивительно для уровня R2 в 0.30

Для региона 1 результаты заметно лучше - RMSE на уровне 1% от среднего значения, т.е. модель предсказывает результаты лучше.

## Подготовка к расчёту прибыли

Зафиксируем все исходные данные:

In [11]:
n_all = 500
n_top = 200
budget = 10000000000
barrel_price = 450000

In [12]:
#рассчитаем достаточный объем сырья для окупаемости:
n_barrel = budget / barrel_price / n_top
print ('Для окупаемости инвестиций в регион из одной скважины в среднем необходимо добывать', n_barrel, 'тысяч баррелей')

Для окупаемости инвестиций в регион из одной скважины в среднем необходимо добывать 111.11111111111111 тысяч баррелей


In [13]:
def comparison_geo(df):
    if df['product'].mean() >= n_barrel:
        return 'Добыча в регионе, скорее всего, рентабельна'
    return 'Добыча в регионе, скорее всего, не рентабельна'

In [14]:
print ('Регион 0:', comparison_geo(df_0))
print ('Регион 1:', comparison_geo(df_1))
print ('Регион 2:', comparison_geo(df_2))

Регион 0: Добыча в регионе, скорее всего, не рентабельна
Регион 1: Добыча в регионе, скорее всего, не рентабельна
Регион 2: Добыча в регионе, скорее всего, не рентабельна


Такая ситуация связана с тем, что в среднем по региону есть как очень небогатые скважины, так и наоборот. Модель позволит ориентироваться только на богатые ресурсами скважины, что всё-таки сделает добычу рентабельной.

Поработаем с предсказаниями. Необходимо написать функцию рассчета прибыли с учетом выбора наилучших скважин.

In [15]:
def revenue(target, prediction_valid, n_top):
    geo_sorted = prediction_valid.sort_values(ascending=False)
    target = pd.Series(target)
    selected = target[geo_sorted.index][:n_top]
    return barrel_price * selected.sum()




In [16]:
print (revenue(target_valid_0, pd.Series(prediction_valid_0, index=target_valid_0.index), n_top))
print (revenue(target_valid_1, pd.Series(prediction_valid_1, index=target_valid_1.index), n_top))
print (revenue(target_valid_2, pd.Series(prediction_valid_2, index=target_valid_2.index), n_top))

13320826043.13985
12415086696.68151
12710349963.599833


Мы видим, что во всех случаех прибыль превышает 10 миллиардов, которые предполагается вложить в разработку скважин. Однако, необходимо иметь в виду, что сами модели отличаются друг от друга по качеству. Соответственно, у этого предсказания разные риски, в связи с чем далее необходимо проанализировать доверительные интервалы.

## Расчёт прибыли и рисков 

Используем bootstrap и оценим доверительный интервал.

In [17]:
#bootstrap:

def bootstrap_geo(target, prediction_valid):
    state = np.random.RandomState(12345)
    values = []
    count = 0
    for i in range(1000):
        subsample = prediction_valid.sample(n = n_all, replace=True, random_state=state)
        rev = revenue(target, subsample, n_top) - 10000000000
        if rev < 0:
            count += 1
        values.append(rev)   
    values = pd.Series(values)
    lower = values.quantile(0.025) # < напишите код здесь >
    upper =  values.quantile(0.975)# < напишите код здесь >
    mean = values.mean()
    return (mean, lower, upper, count / 1000)

In [18]:
mean, lower, upper, prob = bootstrap_geo(target_valid_0, pd.Series(prediction_valid_0, index=target_valid_0.index))
print ("Средняя прибыль для района 0 - ", mean, 'Нижняя граница доверительного интервала -', lower,'Верхняя - ', upper, prob)

mean, lower, upper, prob = bootstrap_geo(target_valid_1, pd.Series(prediction_valid_1, index=target_valid_1.index))
print ("Средняя прибыль для района 1 - ", mean, 'Нижняя граница доверительного интервала -', lower,'Верхняя - ', upper,  prob)

mean, lower, upper, prob = bootstrap_geo(target_valid_2, pd.Series(prediction_valid_2, index=target_valid_2.index))
print ("Средняя прибыль для района 2 - ", mean, 'Нижняя граница доверительного интервала -', lower,'Верхняя - ', upper,  prob)

Средняя прибыль для района 0 -  396164984.802371 Нижняя граница доверительного интервала - -111215545.89049526 Верхняя -  909766941.5534226 0.069
Средняя прибыль для района 1 -  456045105.786661 Нижняя граница доверительного интервала - 33820509.39898363 Верхняя -  852289453.866036 0.015
Средняя прибыль для района 2 -  404403866.56835735 Нижняя граница доверительного интервала - -163350413.39560106 Верхняя -  950359574.9237995 0.076


Выводы: Нижняя граница доверительного интервала оказалась положительной только для региона 1, что логично, ведь модель обеспечивала наибольшую точность, а значит и надежность. Вероятность получить убыток в первом регионе - 0,015. Таким образом, целесообразно производить разработку месторождений в регионе 1.



## Выводы по проекту

В проекте получены следующие результаты:
- Загружены и подготовлены данные
- Данные разбиты на обучающую и валидационную выборки, обучены модели (в качестве основной принята модель линейной регрессии)
- Рассчитано минимальное среднее количество продукта в месторождениях регионов, определена рентабельность разработки (судя по среднему значению, разработка не будет рентабельна ни в одном регионе)
- Проведена оценка рисков: с помощью процедуры Bootstrap получена выборка средних значений, в результате чего оценен доверительный интервал, установивший, что для первого региона вероятность того, что разработка будет нерентабельна - 0,015. В остальных регионах риски слишком высоки. Таким образом, обоснован выбор региона - регион № 1.